In [11]:
import shutil
import pandas as pd
import os
from src.project_definitions import dir

In [2]:
class MasterBrick:
    def __init__(self, name, UID):
        self.Name = name
        self.UIDid = None
        self.UID = UID

        self.A = None
        self.AZ = None

        self.B = None
        self.BZ = None

        self.C = None
        self.CZ = None

        self.D = None
        self.DZ = None
    

        self.mainslots = ['A','B','C','D']
        self.slots = ['A','AZ','B','BZ','C','CZ','D','DZ']

        self.Isolator = []

        self.Note = ''

    def __repr__(self):
        i = 0
        if self.A: i+=1
        if self.B: i+=1
        if self.C: i+=1
        if self.D: i+=1
        return f'MasterBrick {self.Name}: {i} Sensors connected!'

    def connect(self, brick, slot=None, status='online', print_on_connect=False):
        while slot:
            a = getattr(self,slot)
            if getattr(self,slot):
                if a.Name == brick.Name and a.UID == brick.UID:
                    print(f'MasterBrick {self.Name}: {brick.Name} already connected')
                    return
                print(f'MasterBrick {self.Name}: {slot} belegt, verbinde mit nächstem freien Slot.')
                slot = None
                continue
            else:
                setattr(self,slot,brick)
                setattr(brick,'Status',status)
                if len(slot) > 1:
                    if (getattr(self,slot[0]).Type) != 'iso':
                        print(f'MasterBrick {self.Name}: Achtung! Übergeordneter Sensor muss Isolator-Bricklet seit!')
                if print_on_connect:
                    print(f'MasterBrick {self.Name}: {brick.Type}-Sensor connected to Slot {slot}')
                return

        else:
            for slotid in self.mainslots:
                connected = False
                slot = getattr(self,slotid)
                if slot:
                    if slot.Name == brick.Name and slot.UID == brick.UID:
                        print('MasterBrick {self.Name}: {brick.type}-Sensor bereits verbunden.')
                        return
                else: 
                    setattr(self, slotid, brick)
                    setattr(brick,'Status',status)
                    connected = True
                    if print_on_connect:
                        print(f'MasterBrick {self.Name}: {brick.Type}-Sensor connected to Slot {slotid}')
                    return
            if connected == False:
                print('MasterBrick {self.Name}: Bricklet konnte nicht verbunden werden, keine freien Slots verfügbar')

    def overview(self):
        df = pd.DataFrame(columns=['SensorID','Name', 'UID', 'Type', 'Status', 'Note'])
        df.index = pd.MultiIndex(levels=[[],[],[],[]],codes = [[],[],[],[]], names=['MB_Name','MB_UID','MB_NOTE', 'SLOT'])
        for id in self.slots:
            if getattr(self, id): 
                df.loc[(self.Name, self.UID, self.Note, id),:] = getattr(self, id)()

        return df

    def set_note(self,note, slot=None,UID=None):
        if slot:
            brick = getattr(self,slot)
            if brick:
                brick.Note = note
        if UID:
            for brick in[getattr(self,slot) for slot in self.slots]:
                if brick:
                    if brick.UID == UID:
                        brick.Note = note

    def set_slot_ids(self, i):
        for id in self.slots:
            if len(id) == 1:
                i += 1
            if getattr(self,id):
                if len(id) > 1:
                    setattr(getattr(self,id),'SensorID', f'{i}iso')
                else:
                    setattr(getattr(self,id),'SensorID', i)
                #print(f'success! {i}')
        return i
            
class Bricklet:
    def __init__(self, UID, name, type):
        self.Name = name
        self.UID = UID
        self.Type = type
        self.SensorID = None
        self.Status = None
        self.Desc = ''
        self.Note = ''
        self.Bui = ''
        self.Loc = ''
        self.App = ''
        self.Room = ''
        self.Wall = ''
        self.Values = []
    def __call__(self):
        return self.__dict__

In [3]:
# ----TEST ----

MB = MasterBrick('MB20',"5WrSHp")

MB.connect(Bricklet( "Q6c",'MW_2OG_S_WZ_s_reed_Fganz', 'io'),'B','offline')
MB.connect(Bricklet( "MP3",'MW_2OG_S_WZ_s_md_Anw', 'md'))
MB.connect(Bricklet( "V9d",'MW_2OG_S_WZ_pt_Tsk', 'pt_Tsk'),'AZ')

MB.set_slot_ids(10)

MB.overview()

MasterBrick MB20: Achtung! Übergeordneter Sensor muss Isolator-Bricklet seit!


SensorID                      Name  UID    Type  \
MB_Name MB_UID MB_NOTE SLOT                                                   
MB20    5WrSHp         A          11      MW_2OG_S_WZ_s_md_Anw  MP3      md   
                       AZ      11iso        MW_2OG_S_WZ_pt_Tsk  V9d  pt_Tsk   
                       B          12  MW_2OG_S_WZ_s_reed_Fganz  Q6c      io   

                              Status Note  
MB_Name MB_UID MB_NOTE SLOT                
MB20    5WrSHp         A      online       
                       AZ     online       
                       B     offline

In [4]:
MAUERWERK = {}

MAUERWERK['MB1'] = MasterBrick('MB1', "6s7jQr")

MAUERWERK['MB1'].connect(Bricklet("NFB",'MW_2OG_N_SZ_n_trh_TairRH', 'trh'),'B')
MAUERWERK['MB1'].connect(Bricklet("MdT",'MW_2OG_N_SZ_n_co2_LQ', 'co2'),'C')
MAUERWERK['MB1'].connect(Bricklet("Nn3",'MW_2OG_N_SZ_s_pt_Thk', 'ptc'),'A')
MAUERWERK['MB1'].connect(Bricklet( "S9L",'Isolator', 'iso'),'D')
MAUERWERK['MB1'].connect(Bricklet("Q4P",'MW_2OG_N_SZ_o_reed_Fganz', 'io'), 'DZ')    # 22.02. wieder verbunden

MAUERWERK['MB3'] = MasterBrick('MB3', "646rco")

MAUERWERK['MB3'].connect(Bricklet( "Qi9",'MW_2OG_N_el1_Strom', 'el'),'A')
MAUERWERK['MB3'].connect(Bricklet( "Qjn",'MW_2OG_N_el2_Strom', 'el'),'B')
MAUERWERK['MB3'].connect(Bricklet( "Qk1",'MW_2OG_N_el3_Strom', 'el'),'C')

MAUERWERK['MB4'] = MasterBrick('MB4', '6LDqax')

MAUERWERK['MB4'].connect(Bricklet( "NQn",'MW_2OG_N_WZ_s_trh_TairRH', 'trh'), 'A')
MAUERWERK['MB4'].connect(Bricklet( "S9B",'Isolator', 'iso'), 'B')
MAUERWERK['MB4'].connect(Bricklet( "27gpHX",'MW_2OG_N_WZ_o_reed_Fganz', 'io'), 'BZ')
MAUERWERK['MB4'].connect(Bricklet( "V9w",'MW_2OG_N_SZ_pt_Tsk', 'ptc'), 'D')
MAUERWERK['MB4'].connect(Bricklet( "V3t",'MW_2OG_N_WZ_pt_Tsk', 'ptc'), 'C')

MAUERWERK['MB5'] = MasterBrick('MB5', '5XaSdr')
#MAUERWERK['MB5'].connect(Bricklet( "?????", 'Isolator', 'iso'), 'B', status='offline')
MAUERWERK['MB5'].connect(Bricklet( "TFRKK", 'MW_2OG_N_WZ_n_reed_Fganz', 'io'), 'B') # 22.02. wieder verbunden
MAUERWERK['MB5'].connect(Bricklet( "J8S", 'MW_2OG_N_WZ_n_md_Anw', 'md'),'D')
MAUERWERK['MB5'].connect(Bricklet( "NhP", 'MW_2OG_N_WZ_w_pt_Thk', 'ptc'),'A')

MAUERWERK['MB7'] = MasterBrick('MB7', '6Kve5v')
MAUERWERK['MB7'].connect(Bricklet( "Sa4",'Isolator', 'iso'), 'B')
MAUERWERK['MB7'].connect(Bricklet( "NRG",'MW_2OG_N_K_w_trh_TairRH', 'trh'), 'BZ')
MAUERWERK['MB7'].connect(Bricklet( "Q6b",'MW_2OG_N_K_w_reed_Fganz', 'io'),'A')
MAUERWERK['MB7'].connect(Bricklet( "S7D",'Isolator', 'iso'),'C')
MAUERWERK['MB7'].connect(Bricklet( "2rmauH",'MW_2OG_N_K_n_reed_Fganz', 'io'),'CZ')

MAUERWERK['MB9'] = MasterBrick('MB9', "6Lfyvg")
MAUERWERK['MB9'].connect(Bricklet( "Q5Z",'MW_2OG_N_F_w_reed_Fganz', 'io'), 'D')
MAUERWERK['MB9'].connect(Bricklet( "J9c",'MW_2OG_N_F_w_md_Anw', 'md'),'A')
MAUERWERK['MB9'].connect(Bricklet( "NEB",'MW_Dach_trh1_TairRH', 'trh'), 'C')

MAUERWERK['MB10'] = MasterBrick('MB10', '6RNpv4')
MAUERWERK['MB10'].connect(Bricklet( "NcJ",'MW_2OG_N_B_n_pt_Tair', 'ptc'),'A')
MAUERWERK['MB10'].connect(Bricklet( "QP3",'Isolator', 'iso'),'B')
MAUERWERK['MB10'].connect(Bricklet( "NG1",'MW_2OG_N_B_n_trh_TairRH', 'trh'),'BZ')
MAUERWERK['MB10'].connect(Bricklet( "NfP",'MW_2OG_N_B_n_pt_Thk', 'ptc'),'C')
MAUERWERK['MB10'].connect(Bricklet( "NJA",'MW_Dach_trh2_TairRH', 'trh'),'D')

MAUERWERK['MB12'] = MasterBrick('MB12', '6RN528')
MAUERWERK['MB12'].connect(Bricklet( "SsS",'MW_2OG_O_SWK_s_pt_Thk', 'ptc'),'A')
MAUERWERK['MB12'].connect(Bricklet( "S9v",'Isolator', 'iso'),'B')
MAUERWERK['MB12'].connect(Bricklet( "PXd9e",'MW_2OG_O_SWK_o_reed_Fganz', 'io'),'BZ')

MAUERWERK['MB14a'] = MasterBrick('MB14a', '64thHw')
MAUERWERK['MB14a'].connect(Bricklet( "Ryp",'MW_2OG_O_SWK_m_tir1_Twd', 'tir'),'A')
MAUERWERK['MB14a'].set_note('Wall West _ Kitchen', slot='A')
MAUERWERK['MB14a'].connect(Bricklet( "Rw6",'MW_2OG_O_SWK_m_tir2_Twd', 'tir'),'B')
MAUERWERK['MB14a'].set_note('Wall East _ Window', slot='B')
MAUERWERK['MB14a'].connect(Bricklet( "Rwt",'MW_2OG_O_SWK_m_tir3_Twd', 'tir'),'C')
MAUERWERK['MB14a'].set_note('Wall South _ App S', slot='C')
MAUERWERK['MB14a'].connect(Bricklet( "Rxo",'MW_2OG_O_SWK_m_tir4_Twd', 'tir'),'D')
MAUERWERK['MB14a'].set_note('Wall North _ Stair case', slot='D')

MAUERWERK['MB14b'] = MasterBrick('MB14b', '6R3HnK')
MAUERWERK['MB14b'].connect(Bricklet( "RyE",'MW_2OG_O_SWK_m_tir5_Twd', 'tir'),'A')
MAUERWERK['MB14b'].set_note('Ceiling', slot='A')
MAUERWERK['MB14b'].connect(Bricklet( "Rwe",'MW_2OG_O_SWK_m_tir6_Twd', 'tir'),'B')
MAUERWERK['MB14b'].set_note('Floor', slot='B')
MAUERWERK['MB14b'].connect(Bricklet( "NRC",'MW_2OG_O_SWK_m_trh_TairRH', 'trh'),'C')
MAUERWERK['MB14b'].connect(Bricklet( "Stb",'MW_2OG_O_SWK_m_pt_Tsk', 'ptc'),'D')

MAUERWERK['MB14c'] = MasterBrick('MB14c', '6R455i')
MAUERWERK['MB14c'].connect(Bricklet( "McA",'MW_2OG_O_SWK_m_co2_LQ', 'co2'),'A')
MAUERWERK['MB14c'].connect(Bricklet( "Pah",'MW_2OG_O_SWK_m_al_BS', 'al'),'B')
MAUERWERK['MB14c'].connect(Bricklet( "J8J",'MW_2OG_O_SWK_m_md_Anw', 'md'),'C')
MAUERWERK['MB14c'].connect(Bricklet( "FEo",'MW_2OG_O_SWK_m_rtc_Zeit', 'rtc'),'D')

MAUERWERK['MB15'] = MasterBrick('MB15', '6KT3t4')
MAUERWERK['MB15'].connect(Bricklet( "Sti",'MW_2OG_O_B_n_pt_Tair', 'ptc'),'B')
MAUERWERK['MB15'].connect(Bricklet( "QP3",'Isolator', 'iso'),'C')
MAUERWERK['MB15'].connect(Bricklet( "NH4",'MW_2OG_O_B_n_trh_TairRH', 'trh'),'CZ')
MAUERWERK['MB15'].connect(Bricklet( "Ssr",'MW_2OG_O_B_w_pt_Thk', 'ptc'),'A')
MAUERWERK['MB15'].connect(Bricklet( "NJd",'MW_2OG_TH_m_trh_TairRH', 'trh'),'D')

MAUERWERK['MB16'] = MasterBrick('MB16', '64sTzJ')
MAUERWERK['MB16'].connect(Bricklet( "NSG",'MW_2OG_S_SZ_n_trh_TairRH', 'trh'),'B')
MAUERWERK['MB16'].connect(Bricklet( "Me8",'MW_2OG_S_SZ_n_co2_LQ', 'co2'),'A')
MAUERWERK['MB16'].connect(Bricklet( "NeF",'MW_2OG_S_SZ_o_pt_Thk', 'ptc'),'C')
MAUERWERK['MB16'].connect(Bricklet( "V8J",'MW_2OG_S_SZ_pt_Tsk', 'ptc'),'D')

MAUERWERK['MB17'] = MasterBrick('MB17', '6JKEyz')
MAUERWERK['MB17'].connect(Bricklet( '23YXF4','MW_2OG_S_SZ_o_reed_Fganz', 'io'),'A')

MAUERWERK['MB18'] = MasterBrick('MB18', '69EgDB')
MAUERWERK['MB18'].connect(Bricklet( "QjU",'MW_2OG_S_el1_Strom', 'el'),'A')
MAUERWERK['MB18'].connect(Bricklet( "Qj2",'MW_2OG_S_el2_Strom', 'el'),'C')
MAUERWERK['MB18'].connect(Bricklet( "QjQ",'MW_2OG_S_el3_Strom', 'el'),'B')

MAUERWERK['MB19'] = MasterBrick('MB19', '6KS97n')
MAUERWERK['MB19'].connect(Bricklet( "QMx",'Isolator', 'iso'),'A')
MAUERWERK['MB19'].connect(Bricklet( "NRW",'MW_2OG_S_WZ_n_trh_TairRH', 'trh'),'AZ')
MAUERWERK['MB19'].connect(Bricklet( "2n3qWZ",'MW_2OG_S_WZ_o_reed_Fganz', 'io'),'B')

MAUERWERK['MB20'] = MasterBrick('MB20', '5WrSHp')
MAUERWERK['MB20'].connect(Bricklet( "Q6c",'MW_2OG_S_WZ_s_reed_Fganz', 'io'),'B')
MAUERWERK['MB20'].connect(Bricklet( "MP3",'MW_2OG_S_WZ_s_md_Anw', 'md'),'A')
MAUERWERK['MB20'].connect(Bricklet( "V9d",'MW_2OG_S_WZ_pt_Tsk', 'ptc'),'C')

MAUERWERK['MB21'] = MasterBrick('MB21', '69EC3v')
MAUERWERK['MB21'].connect(Bricklet( "NfQ",'MW_2OG_S_WZ_w_pt_Thk', 'ptc'),'A')

MAUERWERK['MB22'] = MasterBrick('MB22', '6tcGsD')
MAUERWERK['MB22'].connect(Bricklet( "ZBNT8",'MW_2OG_S_K_w_reed_Fganz', 'io'),'A')
MAUERWERK['MB22'].connect(Bricklet( "NER",'MW_2OG_S_K_w_trh_TairRH', 'trh'),'B')

MAUERWERK['MB23'] = MasterBrick('MB23', '645R2B')
MAUERWERK['MB23'].connect(Bricklet( "2csnda",'MW_2OG_S_K_s_reed_Fganz', 'io'),'A')

MAUERWERK['MB24'] = MasterBrick('MB24', '6JMZHt')
MAUERWERK['MB24'].connect(Bricklet( "Q5S",'MW_2OG_S_F_w_reed1_Fganz', 'io'),'A')

MAUERWERK['MB25'] = MasterBrick('MB25', '64sXeA')
MAUERWERK['MB25'].connect(Bricklet( "Q4Z",'MW_2OG_S_F_w_reed2_Fganz', 'io'),'B')
MAUERWERK['MB25'].connect(Bricklet( "MNG",'MW_2OG_S_F_n_md_Anw', 'md'),'C')

MAUERWERK['MB26'] = MasterBrick('MB26', '6m9gDk')
MAUERWERK['MB26'].connect(Bricklet( "Njm",'MW_2OG_S_B_n_pt_Tair', 'ptc'),'B')
MAUERWERK['MB26'].connect(Bricklet( "S9G",'Isolator', 'iso'),'C')
MAUERWERK['MB26'].connect(Bricklet( "NFL",'MW_2OG_S_B_s_trh_TairRH', 'trh'),'CZ')
MAUERWERK['MB26'].connect(Bricklet( "Njz",'MW_2OG_S_B_n_pt_Thk', 'ptc'),'A')

MAUERWERK['MB27'] = MasterBrick('MB27', '69Ggjz')
MAUERWERK['MB27'].connect(Bricklet( "Qi7",'MW_2OG_O_el1_Strom', 'el'),'A')
MAUERWERK['MB27'].connect(Bricklet( "Qj1",'MW_2OG_O_el2_Strom', 'el'),'C')
MAUERWERK['MB27'].connect(Bricklet( "QiG",'MW_2OG_O_el3_Strom', 'el'),'B')

MAUERWERK['MB28'] = MasterBrick('MB28', '6F3hiX')
MAUERWERK['MB28'].connect(Bricklet( "CLD",'MW_2OG_O_SWK_al1_BS', 'al'),'A')
MAUERWERK['MB28'].connect(Bricklet( "Pca",'MW_2OG_O_SWK_al2_BS', 'al'),'B')
MAUERWERK['MB28'].connect(Bricklet( "WRM",'MW_2OG_O_SWK_al3_BS', 'al'),'C')
MAUERWERK['MB28'].connect(Bricklet( "P32",'MW_2OG_O_SWK_al4_BS', 'al'),'D')

MAUERWERK['MB29'] = MasterBrick('MB29', '5Xc5Qr')
MAUERWERK['MB29'].connect(Bricklet( "PbC",'MW_2OG_O_SWK_al5_BS', 'al'),'B')
MAUERWERK['MB29'].connect(Bricklet( "NSh",'MW_2OG_O_SWK_m_trh1_TairRH', 'trh'),'A')
MW = []
i=0
for name, brick in MAUERWERK.items():
    if brick:
        i = brick.set_slot_ids(i)
        MW.append(brick.overview())
MW = pd.concat(MW)
MW

SensorID                        Name  UID Type  \
MB_Name MB_UID MB_NOTE SLOT                                                  
MB1     6s7jQr         A           1        MW_2OG_N_SZ_s_pt_Thk  Nn3  ptc   
                       B           2    MW_2OG_N_SZ_n_trh_TairRH  NFB  trh   
                       C           3        MW_2OG_N_SZ_n_co2_LQ  MdT  co2   
                       D           4                    Isolator  S9L  iso   
                       DZ       4iso    MW_2OG_N_SZ_o_reed_Fganz  Q4P   io   
...                              ...                         ...  ...  ...   
MB28    6F3hiX         B          98         MW_2OG_O_SWK_al2_BS  Pca   al   
                       C          99         MW_2OG_O_SWK_al3_BS  WRM   al   
                       D         100         MW_2OG_O_SWK_al4_BS  P32   al   
MB29    5Xc5Qr         A         101  MW_2OG_O_SWK_m_trh1_TairRH  NSh  trh   
                       B         102         MW_2OG_O_SWK_al5_BS  PbC   al   

                             Status Note  
MB_Name MB_UID MB_NOTE SLOT               
MB1     6s7jQr         A     online       
                       B     online       
                       C     online       
                       D     online       
                       DZ    online       
...                             ...  ...  
MB28    6F3hiX         B     online       
                       C     online       
                       D     online       
MB29    5Xc5Qr         A     online       
                       B     online       

[83 rows x 6 columns]

In [5]:
MASSIVHOLZ = {}
mb = 'MB1'
MASSIVHOLZ[mb] = MasterBrick(mb, "6QGzGv")
MASSIVHOLZ[mb].Note =  'old: 6KU7yH ---> siehe MB4b'
MASSIVHOLZ[mb].connect(Bricklet( "NJc",'HM_2OG_N_SZ_n_trh_TairRH', 'trh'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "Mcp",'HM_2OG_N_SZ_n_co2_LQ', 'co2'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "Nrs",'HM_2OG_N_SZ_s_pt_Thk', 'ptc'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "V3c",'HM_2OG_N_SZ_s_pt_Thk', 'ptc'),'D')

mb = 'MB2'
MASSIVHOLZ[mb] = MasterBrick(mb, "5Xat5D")
MASSIVHOLZ[mb].connect(Bricklet( "2CEa12",'HM_2OG_N_SZ_o_reed_Fganz', 'io'),'A')

mb = 'MB3'
MASSIVHOLZ[mb] = MasterBrick(mb, "6RMJxc")
MASSIVHOLZ[mb].connect(Bricklet( "QjJ",'HM_2OG_N_el1_Strom', 'al'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "Qic",'HM_2OG_N_el2_Strom', 'al'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "Qi4",'HM_2OG_N_el3_Strom', 'al'),'C')

mb = 'MB4'
MASSIVHOLZ[mb] = MasterBrick(mb, "6KS7TK")
MASSIVHOLZ[mb].connect(Bricklet( "Sa5",'Isolator', 'iso'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "NRH",'HM_2OG_N_WZ_s_trh_TairRH', 'trh'),'BZ')
MASSIVHOLZ[mb].connect(Bricklet( "29dB1H",'HM_2OG_N_WZ_o_reed_Fganz', 'io'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "V3c",'HM_2OG_N_WZ_pt_Tsk', 'ptc'),'C',status='offline')    #Nicht vorhanden
MASSIVHOLZ[mb].connect(Bricklet( "V9C",'HM_2OG_N_WZ_pt_Tsk', 'ptc'),'D')

mb = 'MB5'
MASSIVHOLZ[mb] = MasterBrick(mb, "5Xa8AH")
MASSIVHOLZ[mb].connect(Bricklet( "U8dBx",'HM_2OG_N_WZ_n_reed_Fganz', 'io'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "Rn7",'HM_2OG_N_WZ_n_md_Anw', 'md'),'B')

mb = 'MB6'
MASSIVHOLZ[mb] = MasterBrick(mb, "6gm7LR")
MASSIVHOLZ[mb].connect(Bricklet( "Nf7",'HM_2OG_N_WZ_w_pt_Thk', 'ptc'),'A')

mb = 'MB7'
MASSIVHOLZ[mb] = MasterBrick(mb, "6F4YWe")
MASSIVHOLZ[mb].connect(Bricklet( "S9F",'Isolator', 'iso'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "NHa",'HM_2OG_N_K_w_trh_TairRH', 'trh'),'AZ')
MASSIVHOLZ[mb].connect(Bricklet( "29egYz",'HM_2OG_N_K_w_reed_Fganz', 'io'),'B')

mb = 'MB8'
MASSIVHOLZ[mb] = MasterBrick(mb, "5Xb93v")
MASSIVHOLZ[mb].connect(Bricklet( "RrwUv",'HM_2OG_N_K_n_reed_Fganz', 'io'),'A')

mb = 'MB9'
MASSIVHOLZ[mb] = MasterBrick(mb, "6su43R")
MASSIVHOLZ[mb].connect(Bricklet( "2ckDzx",'HM_2OG_N_F_w_reed_Fganz', 'io'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "MNC",'HM_2OG_N_F_w_md_Anw', 'md'),'A')

mb = 'MB10'
MASSIVHOLZ[mb] = MasterBrick(mb, "6RQqoD")
MASSIVHOLZ[mb].connect(Bricklet( "NqU",'HM_2OG_N_B_n_pt_Tair', 'ptc'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "NFa",'HM_2OG_N_B_n_trh_TairRH', 'trh'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "Rrz",'HM_2OG_N_B_n_pt_Thk', 'ptc'),'A')

mb = 'MB11'
MASSIVHOLZ[mb] = MasterBrick(mb, "69DgcP")
MASSIVHOLZ[mb].connect(Bricklet( "NGH",'HM_2OG_TH_m_trh_TairRH', 'trh'),'A')

mb = 'MB12'
MASSIVHOLZ[mb] = MasterBrick(mb, "69j1ED")
MASSIVHOLZ[mb].connect(Bricklet( "RrR",'HM_2OG_O_SWK_s_pt_Thk', 'ptc'),'A')

mb = 'MB13'
MASSIVHOLZ[mb] = MasterBrick(mb, "6RPoJe")
MASSIVHOLZ[mb].connect(Bricklet( "PLi",'HM_2OG_O_SWK_o_reed_Fganz', 'io'),'A')

mb = 'MB14a'
MASSIVHOLZ[mb] = MasterBrick(mb, "6DWu2F")
MASSIVHOLZ[mb].connect(Bricklet( "FzG",'HM_2OG_O_SWK_m_rtc_Zeit', 'rtc'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "MNH",'HM_2OG_O_SWK_m_md_Anw', 'md'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "NJP",'HM_2OG_O_SWK_m_trh_TairRH', 'trh'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "RrJ",'HM_2OG_O_SWK_m_pt_Tsk', 'ptc'),'D')

mb = 'MB14b'
MASSIVHOLZ[mb] = MasterBrick(mb, "69DXoi")
MASSIVHOLZ[mb].connect(Bricklet( "RwG",'HM_2OG_O_SWK_m_tir2_Twd', 'tir'),'A')
MASSIVHOLZ[mb].set_note('Wall South _ App S', slot='A')
MASSIVHOLZ[mb].connect(Bricklet( "Rx6",'HM_2OG_O_SWK_m_tir4_Twd', 'tir'),'B')
MASSIVHOLZ[mb].set_note(' Wall North _ Stair case', slot='B')
MASSIVHOLZ[mb].connect(Bricklet( "Ryi",'HM_2OG_O_SWK_m_tir3_Twd', 'tir'),'C')
MASSIVHOLZ[mb].set_note('Wall East _ Window', slot='C')
MASSIVHOLZ[mb].connect(Bricklet( "Ryy",'HM_2OG_O_SWK_m_tir1_Twd', 'tir'),'D')
MASSIVHOLZ[mb].set_note('Wall West _ Kitchen', slot='D')

mb = 'MB14c'
MASSIVHOLZ[mb] = MasterBrick(mb, "6DVtzT")
MASSIVHOLZ[mb].connect(Bricklet( "Me7",'HM_2OG_O_SWK_m_co2_LQ', 'co2'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "P4U",'HM_2OG_O_SWK_m_al_BS', 'al'),'D')
MASSIVHOLZ[mb].connect(Bricklet( "RyB",'HM_2OG_O_SWK_m_tir5_Twd', 'tir'),'A')
MASSIVHOLZ[mb].set_note('Ceiling', slot='A')
MASSIVHOLZ[mb].connect(Bricklet( "Ryb",'HM_2OG_O_SWK_m_tir6_Twd', 'tir'),'B')
MASSIVHOLZ[mb].set_note('Floor', slot='B')

mb = 'MB15'
MASSIVHOLZ[mb] = MasterBrick(mb, "6KSw2x")
MASSIVHOLZ[mb].connect(Bricklet( "NmZ",'HM_2OG_O_B_n_pt_Tair', 'ptc'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "S9r",'Isolator', 'iso'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "NHz",'HM_2OG_O_B_n_trh_TairRH', 'trh'),'CZ')
MASSIVHOLZ[mb].connect(Bricklet( "RrS",'HM_2OG_O_B_w_pt_Thk', 'ptc'),'B')

mb = 'MB16'
MASSIVHOLZ[mb] = MasterBrick(mb, "6fAUXt")
MASSIVHOLZ[mb].connect(Bricklet( "NRE",'HM_2OG_S_SZ_n_trh_TairRH', 'trh'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "Mda",'HM_2OG_S_SZ_n_co2_LQ', 'co2'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "Rsc",'HM_2OG_S_SZ_o_pt_Thk', 'ptc'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "V9z",'HM_2OG_S_SZ_pt_Tsk', 'ptc'),'D')

mb = 'MB17'
MASSIVHOLZ[mb] = MasterBrick(mb, "6RP4fi")
MASSIVHOLZ[mb].connect(Bricklet( "2qvhVK",'HM_2OG_S_SZ_o_reed_Fganz', 'io'),'A')

mb = 'MB18'
MASSIVHOLZ[mb] = MasterBrick(mb, "6KT8kx")
MASSIVHOLZ[mb].connect(Bricklet( "QjP",'HM_2OG_S_el1_Strom', 'al'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "Qj5",'HM_2OG_S_el2_Strom', 'al'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "QiP",'HM_2OG_S_el3_Strom', 'al'),'C')

mb = 'MB19'
MASSIVHOLZ[mb] = MasterBrick(mb, "647NyZ")
MASSIVHOLZ[mb].connect(Bricklet( "NGp",'HM_2OG_S_WZ_n_trh_TairRH', 'trh'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "2xVJyn",'HM_2OG_S_WZ_o_reed_Fganz', 'io'),'B')

mb = 'MB20'
MASSIVHOLZ[mb] = MasterBrick(mb, "69FfSM")
MASSIVHOLZ[mb].connect(Bricklet( "2hFvjV",'HM_2OG_S_WZ_s_reed_Fganz', 'io'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "MMN",'HM_2OG_S_WZ_s_md_Anw', 'md'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "V8L",'HM_2OG_S_WZ_pt_Tsk', 'ptc'),'C')

mb = 'MB21'
MASSIVHOLZ[mb] = MasterBrick(mb, "6DXMwa")
MASSIVHOLZ[mb].connect(Bricklet( "Ngu",'HM_2OG_S_WZ_w_pt_Thk', 'ptc'),'A')

mb = 'MB22'
MASSIVHOLZ[mb] = MasterBrick(mb, "6RQ4G6")
MASSIVHOLZ[mb].connect(Bricklet( "VCdkF",'HM_2OG_S_K_w_reed_Fganz', 'io'),'C')
MASSIVHOLZ[mb].connect(Bricklet( "NSo",'HM_2OG_S_K_w_trh_TairRH', 'trh'),'A')

mb = 'MB23'
MASSIVHOLZ[mb] = MasterBrick(mb, "6DWM5n")
MASSIVHOLZ[mb].connect(Bricklet( "2iojkx",'HM_2OG_S_K_s_reed_Fganz', 'io'),'A')

mb = 'MB24'
MASSIVHOLZ[mb] = MasterBrick(mb, "6KUtgg")
MASSIVHOLZ[mb].connect(Bricklet( "271CwZ",'HM_2OG_S_F_w_reed1_Fganz', 'io'),'A')

mb = 'MB25'
MASSIVHOLZ[mb] = MasterBrick(mb, "6KRt9v")
MASSIVHOLZ[mb].connect(Bricklet( "2cSo9e",'HM_2OG_S_F_w_reed2_Fganz', 'io'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "MMD",'HM_2OG_S_F_n_md_Anw', 'md'),'B')

mb = 'MB26'
MASSIVHOLZ[mb] = MasterBrick(mb, "6463hR")
MASSIVHOLZ[mb].connect(Bricklet( "NkH",'HM_2OG_S_B_n_pt_Tair', 'al'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "NQT",'HM_2OG_S_B_s_trh_TairRH', 'al'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "RrZ",'HM_2OG_S_B_n_pt_Thk', 'al'),'C')

mb = 'MB27'
MASSIVHOLZ[mb] = MasterBrick(mb, "6nh4Ue")
MASSIVHOLZ[mb].connect(Bricklet( "QiC",'HM_2OG_O_el1_Strom', 'al'),'A')
MASSIVHOLZ[mb].connect(Bricklet( "QjS",'HM_2OG_O_el2_Strom', 'al'),'B')
MASSIVHOLZ[mb].connect(Bricklet( "QiF",'HM_2OG_O_el3_Strom', 'al'),'C')

mb = 'MB27'
MASSIVHOLZ[mb] = MasterBrick(mb, "6LfVcP")
MASSIVHOLZ[mb].connect(Bricklet( "NSn",'HM_2OG_O_SWK_m_trh1_TairRH', 'trh'),'A')

MH = []
i=0
for name, brick in MASSIVHOLZ.items():
    if brick:
        i = brick.set_slot_ids(i)
        MH.append(brick.overview())
MH = pd.concat(MH)
MH

SensorID  \
MB_Name MB_UID MB_NOTE                     SLOT            
MB1     6QGzGv old: 6KU7yH ---> siehe MB4b A           1   
                                           B           2   
                                           C           3   
                                           D           4   
MB2     5Xat5D                             A           5   
...                                                  ...   
MB25    6KRt9v                             B         106   
MB26    6463hR                             A         109   
                                           B         110   
                                           C         111   
MB27    6LfVcP                             A         113   

                                                                       Name  \
MB_Name MB_UID MB_NOTE                     SLOT                               
MB1     6QGzGv old: 6KU7yH ---> siehe MB4b A           HM_2OG_N_SZ_s_pt_Thk   
                                           B       HM_2OG_N_SZ_n_trh_TairRH   
                                           C           HM_2OG_N_SZ_n_co2_LQ   
                                           D           HM_2OG_N_SZ_s_pt_Thk   
MB2     5Xat5D                             A       HM_2OG_N_SZ_o_reed_Fganz   
...                                                                     ...   
MB25    6KRt9v                             B            HM_2OG_S_F_n_md_Anw   
MB26    6463hR                             A           HM_2OG_S_B_n_pt_Tair   
                                           B        HM_2OG_S_B_s_trh_TairRH   
                                           C            HM_2OG_S_B_n_pt_Thk   
MB27    6LfVcP                             A     HM_2OG_O_SWK_m_trh1_TairRH   

                                                    UID Type  Status Note  
MB_Name MB_UID MB_NOTE                     SLOT                            
MB1     6QGzGv old: 6KU7yH ---> siehe MB4b A        Nrs  ptc  online       
                                           B        NJc  trh  online       
                                           C        Mcp  co2  online       
                                           D        V3c  ptc  online       
MB2     5Xat5D                             A     2CEa12   io  online       
...                                                 ...  ...     ...  ...  
MB25    6KRt9v                             B        MMD   md  online       
MB26    6463hR                             A        NkH   al  online       
                                           B        NQT   al  online       
                                           C        RrZ   al  online       
MB27    6LfVcP                             A        NSn  trh  online       

[68 rows x 6 columns]

In [6]:
LEICHTBETON = {}

mb = 'MB1'
LEICHTBETON[mb] = MasterBrick(mb, "5XbP1n") #
LEICHTBETON[mb].connect(Bricklet( "RHF",'LB_2OG_N_SZ_n_trh_TairRH', 'trh'),'C')
LEICHTBETON[mb].connect(Bricklet( "C2C",'LB_2OG_N_SZ_n_co2_LQ', 'co2'),'B')
LEICHTBETON[mb].connect(Bricklet( "RrX",'LB_2OG_N_SZ_s_pt_Thk', 'ptc'),'D')
LEICHTBETON[mb].connect(Bricklet( "TF",'LB_2OG_N_SZ_s_t_Tair', 'temp'),'A')
mb = 'MB2'
LEICHTBETON[mb] = MasterBrick(mb, "6SUfdz") #
LEICHTBETON[mb].connect(Bricklet( "27TFJv",'LB_2OG_N_SZ_o_reed_Fganz', 'trh'),'A')
LEICHTBETON[mb].connect(Bricklet( "BNSZ",'??', 'ptc'),'B')
LEICHTBETON[mb].connect(Bricklet( "NR8",'LB_Dach_trh1_TairRH', 'trh'),'C')
mb = 'MB3'
LEICHTBETON[mb] = MasterBrick(mb, "5XwVsZ") #
LEICHTBETON[mb].connect(Bricklet( "ELD",'LB_2OG_N_el1_Strom', 'el'),'A')
LEICHTBETON[mb].connect(Bricklet( "ELF",'LB_2OG_N_el2_Strom', 'el'),'B')
LEICHTBETON[mb].connect(Bricklet( "Qj4",'LB_2OG_N_el3_Strom', 'el'),'C')
mb = 'MB4'
LEICHTBETON[mb] = MasterBrick(mb, "6Lg15i")
LEICHTBETON[mb].Note = 'changed from "6KUqQ2"'
LEICHTBETON[mb].connect(Bricklet( "RHG",'LB_2OG_N_WZ_s_trh_TairRH', 'trh'),'D')
LEICHTBETON[mb].connect(Bricklet( "TG",'LB_2OG_N_WZ_s_t_Tair', 'temp'),'A','offline')
LEICHTBETON[mb].connect(Bricklet( "BNWZ",'LB_2OG_N_WZ_pt_Tsk', 'ptc'),'C')
LEICHTBETON[mb].connect(Bricklet( "QNZ",'Isolator', 'iso'),'B')
LEICHTBETON[mb].connect(Bricklet( "NSa",'LB_2OG_N_WZ_m_trh_TairRH', 'trh'),'BZ')
mb = 'MB4b'
LEICHTBETON[mb] = MasterBrick(mb, "6K9pdM") #
LEICHTBETON[mb].connect(Bricklet( "2acnp4",'LB_2OG_N_WZ_o_reed_Fganz', 'io'),'C')
mb = 'MB5'
LEICHTBETON[mb] = MasterBrick(mb, "6469nX") #
LEICHTBETON[mb].connect(Bricklet( "2i7GrK",'LB_2OG_N_WZ_n_reed_Fganz', 'io'),'B')
LEICHTBETON[mb].connect(Bricklet( "MDD",'LB_2OG_N_WZ_n_md_Anw', 'md'),'A')
mb = 'MB6'
LEICHTBETON[mb] = MasterBrick(mb, "5WN48x") #
LEICHTBETON[mb].connect(Bricklet( "Nkp",'LB_2OG_N_WZ_w_pt_Thk', 'ptc'),'A')
mb = 'MB7'
LEICHTBETON[mb] = MasterBrick(mb, "6DXPXp") #
LEICHTBETON[mb].connect(Bricklet( "RHK",'LB_2OG_N_K_w_trh_TairRH', 'trh'),'A')
LEICHTBETON[mb].connect(Bricklet( "TK",'LB_2OG_N_K_w_t_Tair', 'temp'),'C')
mb = 'MB7b'
LEICHTBETON[mb] = MasterBrick(mb, "5WrQha") #
LEICHTBETON[mb].connect(Bricklet( "2nvitt",'LB_2OG_N_K_w_reed_Fganz', 'io'),'D')
mb = 'MB8'
LEICHTBETON[mb] = MasterBrick(mb, "6RQpb2") #
LEICHTBETON[mb].connect(Bricklet( "2oZsuM",'LB_2OG_N_K_n_reed_Fganz', 'io'),'A')
mb = 'MB9'
LEICHTBETON[mb] = MasterBrick(mb, "63JZu6") #
LEICHTBETON[mb].connect(Bricklet( "23rTCr",'LB_2OG_N_F_w_reed_Fganz', 'io'),'A')
mb = 'MB9b'
LEICHTBETON[mb] = MasterBrick(mb, "6R5Jgk") #
LEICHTBETON[mb].connect(Bricklet( "MDE",'LB_2OG_N_F_w_md_Anw', 'md'),'A')
mb = 'MB10'
LEICHTBETON[mb] = MasterBrick(mb, "6KUbdz") #
LEICHTBETON[mb].connect(Bricklet( "RrU",'LB_2OG_N_B_n_pt_Tair', 'ptc'),'C')
LEICHTBETON[mb].connect(Bricklet( "Sa8",'Isolator', 'iso'),'B')
LEICHTBETON[mb].connect(Bricklet( "RHJ",'LB_2OG_N_B_n_trh_TairRH', 'trh'),'BZ')
LEICHTBETON[mb].connect(Bricklet( "Nnb",'LB_2OG_N_B_n_pt_Thk', 'ptc'),'A')
mb = 'MB11'
LEICHTBETON[mb] = MasterBrick(mb, "6stNrp") #
LEICHTBETON[mb].connect(Bricklet( "RHH",'LB_2OG_TH_m_trh_TairRH', 'trh'),'A')
LEICHTBETON[mb].connect(Bricklet( "TH",'LB_2OG_TH_m_t_Tair', 'temp'),'B')
LEICHTBETON[mb].connect(Bricklet( "N54",'ow_1', 'owb'),'C')
LEICHTBETON[mb].connect(Bricklet( "DAA",'LB_DA_W_id_Pyrano', 'pyrano'),'D')
mb = 'MB12'
LEICHTBETON[mb] = MasterBrick(mb, "64893V") #
LEICHTBETON[mb].connect(Bricklet( "RrN",'LB_2OG_O_SWK_s_pt_Thk', 'ptc'),'C')
mb = 'MB13'
LEICHTBETON[mb] = MasterBrick(mb, "6DWPvB") #
LEICHTBETON[mb].connect(Bricklet( "VLrCZ",'LB_2OG_O_SWK_o_reed_Fganz', 'io'),'A')
mb = 'MB14a'
LEICHTBETON[mb] = MasterBrick(mb, "6a3KoH") #
LEICHTBETON[mb].connect(Bricklet( "RTC",'LB_2OG_O_SWK_m_rtc_Zeit', 'rtc'),'A')
LEICHTBETON[mb].connect(Bricklet( "Sso",'LB_2OG_O_SWK_m_pt_Tsk', 'ptc'),'B')
LEICHTBETON[mb].connect(Bricklet( "TA",'LB_2OG_O_SWK_m_t_Tair', 'temp'),'C')
LEICHTBETON[mb].connect(Bricklet( "TCL",'LB_2OG_O_SWK_m_tcl_Tsk', 'tcl'),'D')
mb = 'MB14b'
LEICHTBETON[mb] = MasterBrick(mb, "6gmbcs") #
LEICHTBETON[mb].connect(Bricklet( "TRA",'LB_2OG_O_SWK_m_tir1_Twd', 'tir'),'A')
LEICHTBETON[mb].set_note('Floor',slot='A')
LEICHTBETON[mb].connect(Bricklet( "TRB",'LB_2OG_O_SWK_m_tir2_Twd', 'tir'),'B')
LEICHTBETON[mb].set_note('Ceiling',slot='B')
LEICHTBETON[mb].connect(Bricklet( "TRC",'LB_2OG_O_SWK_m_tir3_Twd', 'tir'),'C')
LEICHTBETON[mb].set_note('Wall West _ Kitchen',slot='C')
LEICHTBETON[mb].connect(Bricklet( "TRD",'LB_2OG_O_SWK_m_tir4_Twd', 'tir'),'D')
LEICHTBETON[mb].set_note('Wall East _ Window',slot='D')
mb = 'MB14c'
LEICHTBETON[mb] = MasterBrick(mb, "6QHWQR") #
LEICHTBETON[mb].connect(Bricklet( "C2A",'LB_2OG_O_SWK_m_co2_LQ', 'co2'),'D')
LEICHTBETON[mb].connect(Bricklet( "RHA",'LB_2OG_O_SWK_m_trh_TairRH', 'trh'),'C')
LEICHTBETON[mb].connect(Bricklet( "TRE",'LB_2OG_O_SWK_m_tir5_Twd', 'tir'),'A')
LEICHTBETON[mb].set_note('Wall North _ Staircase',slot='A')
LEICHTBETON[mb].connect(Bricklet( "TRF",'LB_2OG_O_SWK_m_tir6_Twd', 'tir'),'B')
LEICHTBETON[mb].set_note('Wall South _ App S',slot='B')
mb = 'MB14d'
LEICHTBETON[mb] = MasterBrick(mb, "6QHi6B") #
LEICHTBETON[mb].connect(Bricklet( "ALA",'LB_2OG_O_SWK_m_al_BS', 'al'),'A')
LEICHTBETON[mb].connect(Bricklet( "MDA",'LB_2OG_O_SWK_m_md_Anw', 'md'),'B')
mb = 'MB15'
LEICHTBETON[mb] = MasterBrick(mb, "6KRR4F") #
LEICHTBETON[mb].connect(Bricklet( "NqR",'LB_2OG_O_B_n_pt_Tair', 'ptc'),'C')
LEICHTBETON[mb].connect(Bricklet( "NRD",'LB_2OG_O_B_n_trh_TairRH', 'trh'),'B')
LEICHTBETON[mb].connect(Bricklet( "RrH",'LB_2OG_O_B_w_pt_Thk', 'ptc'),'A')
mb = 'MB15b'
LEICHTBETON[mb] = MasterBrick(mb, "6Lgkye") #
LEICHTBETON[mb].connect(Bricklet( "QjA",'LB_2OG_O_el1_Strom', 'el'),'A')
LEICHTBETON[mb].connect(Bricklet( "QiH",'LB_2OG_O_el2_Strom', 'el'),'B')
LEICHTBETON[mb].connect(Bricklet( "QiN",'LB_2OG_O_el3_Strom', 'el'),'C')
mb = 'MB16'
LEICHTBETON[mb] = MasterBrick(mb, "6RMp4g") #
LEICHTBETON[mb].connect(Bricklet( "RHB",'LB_2OG_S_SZ_n_trh_TairRH', 'trh'),'B')
LEICHTBETON[mb].connect(Bricklet( "C2B",'LB_2OG_S_SZ_n_co2_LQ', 'co2'),'A')
LEICHTBETON[mb].connect(Bricklet( "SsN",'LB_2OG_S_SZ_o_pt_Thk', 'ptc'),'D')
LEICHTBETON[mb].connect(Bricklet( "TB",'LB_2OG_S_SZ_n_t_Tair', 'temp'),'C')
mb = 'MB17'
LEICHTBETON[mb] = MasterBrick(mb, "6KSbxB") #
LEICHTBETON[mb].connect(Bricklet( "2hod2t",'LB_2OG_S_SZ_o_reed_Fganz', 'io'),'A')
LEICHTBETON[mb].connect(Bricklet( "BSSZ",'LB_2OG_S_SZ_pt_Tsk', 'ptc'),'B')
mb = 'MB18'
LEICHTBETON[mb] = MasterBrick(mb, "6RNJYZ") #
LEICHTBETON[mb].connect(Bricklet( "ELA",'LB_2OG_S_el1_Strom', 'el'),'A')
LEICHTBETON[mb].connect(Bricklet( "ELB",'LB_2OG_S_el2_Strom', 'el'),'B')
LEICHTBETON[mb].connect(Bricklet( "ELC",'LB_2OG_S_el3_Strom', 'el'),'C')
mb = 'MB19'
LEICHTBETON[mb] = MasterBrick(mb, "6RN8EZ") #
LEICHTBETON[mb].connect(Bricklet( "RHC",'LB_2OG_S_WZ_n_trh_TairRH', 'trh'),'A')
LEICHTBETON[mb].connect(Bricklet( "TC",'LB_2OG_S_WZ_n_t_Tair', 'temp'),'B')
mb = 'MB19b'
LEICHTBETON[mb] = MasterBrick(mb, "6LeBHk") #
LEICHTBETON[mb].connect(Bricklet( "2jBwqV",'LB_2OG_S_WZ_o_reed_Fganz', 'io'),'A')
mb = 'MB20'
LEICHTBETON[mb] = MasterBrick(mb, "6KTqoe") #
LEICHTBETON[mb].connect(Bricklet( "24bo3H",'LB_2OG_S_WZ_s_reed_Fganz', 'io'),'A')
LEICHTBETON[mb].connect(Bricklet( "MDB",'LB_2OG_S_WZ_s_md_Anw', 'md'),'B')
LEICHTBETON[mb].connect(Bricklet( "NjT",'LB_2OG_S_WZ_w_pt_Thk', 'ptc'),'C')
LEICHTBETON[mb].connect(Bricklet( "BSWZ",'LB_2OG_S_WZ_pt_Tsk', 'ptc'),'D')
mb = 'MB22'
LEICHTBETON[mb] = MasterBrick(mb, "69DD7Z") #
LEICHTBETON[mb].connect(Bricklet( "RHD",'LB_2OG_S_K_w_trh_TairRH', 'trh'),'B')
LEICHTBETON[mb].connect(Bricklet( "TD",'LB_2OG_S_K_w_t_Tair', 'temp'),'C')
mb = 'MB22b'
LEICHTBETON[mb] = MasterBrick(mb, "5WpRPP") #
LEICHTBETON[mb].connect(Bricklet( "26pSc8",'LB_2OG_S_K_w_reed_Fganz', 'io'),'A')
mb = 'MB23'
LEICHTBETON[mb] = MasterBrick(mb, "6maX3Z") #
LEICHTBETON[mb].connect(Bricklet( "WikWD",'LB_2OG_S_K_s_reed_Fganz', 'io'),'A')
mb = 'MB24'
LEICHTBETON[mb] = MasterBrick(mb, "647Mmn") #
LEICHTBETON[mb].connect(Bricklet( "2j9Zon",'LB_2OG_S_F_w_reed1_Fganz', 'io'),'B')
mb = 'MB25'
LEICHTBETON[mb] = MasterBrick(mb, "63JeDK") #
LEICHTBETON[mb].connect(Bricklet( "2dF55D",'LB_2OG_S_F_w_reed2_Fganz', 'io'),'A')
mb = 'MB25b'
LEICHTBETON[mb] = MasterBrick(mb, "5WqSgB") #
LEICHTBETON[mb].connect(Bricklet( "MDC",'LB_2OG_S_F_n_md_Anw', 'md'),'A')
mb = 'MB26'
LEICHTBETON[mb] = MasterBrick(mb, "6R3DHT") #
LEICHTBETON[mb].connect(Bricklet( "Rs1",'LB_2OG_S_B_n_pt_Tair', 'ptc'),'B')
LEICHTBETON[mb].connect(Bricklet( "RHE",'LB_2OG_S_B_s_trh_TairRH', 'trh'),'C')
LEICHTBETON[mb].connect(Bricklet( "NkE",'LB_2OG_S_B_n_pt_Thk', 'ptc'),'A')
mb = 'MB34'
LEICHTBETON[mb] = MasterBrick(mb, "6s4EcB") #
LEICHTBETON[mb].connect(Bricklet( "NHT",'LB_2OG_O_SWK_m_trh1_TairRH', 'trh'),'A')

LB = []
i=0
for name, brick in LEICHTBETON.items():
    if brick:
        i = brick.set_slot_ids(i)
        LB.append(brick.overview())
LB = pd.concat(LB)
LB

SensorID                        Name     UID  \
MB_Name MB_UID MB_NOTE SLOT                                                
MB1     5XbP1n         A           1        LB_2OG_N_SZ_s_t_Tair      TF   
                       B           2        LB_2OG_N_SZ_n_co2_LQ     C2C   
                       C           3    LB_2OG_N_SZ_n_trh_TairRH     RHF   
                       D           4        LB_2OG_N_SZ_s_pt_Thk     RrX   
MB2     6SUfdz         A           5    LB_2OG_N_SZ_o_reed_Fganz  27TFJv   
...                              ...                         ...     ...   
MB25b   5WqSgB         A         133         LB_2OG_S_F_n_md_Anw     MDC   
MB26    6R3DHT         A         137         LB_2OG_S_B_n_pt_Thk     NkE   
                       B         138        LB_2OG_S_B_n_pt_Tair     Rs1   
                       C         139     LB_2OG_S_B_s_trh_TairRH     RHE   
MB34    6s4EcB         A         141  LB_2OG_O_SWK_m_trh1_TairRH     NHT   

                             Type  Status Note  
MB_Name MB_UID MB_NOTE SLOT                     
MB1     5XbP1n         A     temp  online       
                       B      co2  online       
                       C      trh  online       
                       D      ptc  online       
MB2     6SUfdz         A      trh  online       
...                           ...     ...  ...  
MB25b   5WqSgB         A       md  online       
MB26    6R3DHT         A      ptc  online       
                       B      ptc  online       
                       C      trh  online       
MB34    6s4EcB         A      trh  online       

[82 rows x 6 columns]

In [7]:
Bricklet( "NH2",'LB_Dach_trh2_TairRH', 'ptc')

In [15]:
if not os.path.isfile(os.path.join(dir,'Sensoruebersicht.xlsx')):
    excelfile = '.\src\Sensoruebersicht_template.xlsx'
    shutil.copy(excelfile, os.path.join(dir,'Sensoruebersicht.xlsx'))
    excelfile = os.path.isfile(os.path.join(dir,'Sensoruebersicht.xlsx'))
else:
    excelfile = os.path.isfile(os.path.join(dir,'Sensoruebersicht.xlsx'))
with pd.ExcelWriter(os.path.join(dir,'Sensoruebersicht.xlsx'),mode='a', if_sheet_exists='overlay') as writer:
    LB.to_excel(writer,sheet_name='Leichtbeton (LB)', startrow=24)
    MH.to_excel(writer,sheet_name='Massivholz (MH)', startrow=24)
    MW.to_excel(writer,sheet_name='Mauerwerk (MW)', startrow=24)